# NLM UI Exploration using scverse Pseudo-bulk functional analysis with NSForest

Using a dataset from the `COvid-19 Multi-omics Blood ATlas (COMBAT) Consortium` published in 2022 in Cell `A blood atlas of COVID-19 defines hallmarks of disease severity and specificity`.  We will explore possible user interfaces to answer questions about cell types.

When cell lineage is clear (there are clear cell identity clusters), it might be beneficial to perform functional analyses at the pseudo-bulk level instead of the single-cell.
By doing so, we recover lowly expressed genes that before where affected by the "drop-out" effect of single-cell. 
Additionaly, if there is more than one condition in our data, we can perform differential expression analysis (DEA) and use the gene statistics as input for enrichment analysis.

In this notebook we showcase how to use `decoupler` for pathway and transcription factor (TF) enrichment from a human data-set. The data consists of ~5k Blood myeloid cells from healthy and COVID-19 infected patients available in the Single Cell Expression Atlas [here](https://www.ebi.ac.uk/gxa/sc/experiments/E-MTAB-9221/results?plotType=umap&plotOption=20).

<div class="alert alert-info">

**Note**
    
This notebook assumes that you already know the basics of `decoupler`. Else, check out the [Usage](https://decoupler-py.readthedocs.io/en/latest/notebooks/usage.html) tutorial first.   A version is in `adeslatt` github to allow it to run on MacOS.

</div>  

## for local execution on a MacOS - modifications by adeslatt*

To pull from the cellxgene the appropriate data set, additional packages need to be installed:

```bash
pip install cellxgene-census scanpy scvi-tools
```

The rest of these also need to be preformed.

* create a clean conda env
* install in this environment the required packages that are in the *import* line
* install done 2024-10-20 - needed to restrict python version
* decoupler not available in bioconda at the right version and not present in conda-forge
* pip install decoupler instead

In a bash shell.  If you are already in an environment, be sure to deactivate it to get to your base environment.

install jupyter and kernels of interest

installing scanpy installs numpy 

```bash
conda update -n base -c defaults conda
conda create -n scverse python=3.12.7
conda activate scverse
conda install conda-forge::jupyter -y
conda install conda-forge::ipykernel -y
conda install conda-forge::r-irkernel -y
conda install conda-forge::bash_kernel -y
conda install conda-forge::scanpy -y
pip install decoupler
```

## first run resulted in an error

Return to the bash shell within your conda environment and upgrade the decoupler package.
Other missing packages:
1. pybiomart - the python front end for biomart
2. marsilea
3. pydeseq2 - the python front end to DESeq2
4. igraph
5. adjustText
6. pypath-omnipath


```bash
 pip install pybiomart
 pip install marsilea
 pip install pydeseq2
 pip install igraph
 pip install adjustText
 pip install omnipath
```

The above fixes the problem.


## Using the CELLxGENE Census APIs to pull data

In [1]:
# import the cellxgene_census package
import cellxgene_census


In [2]:
# set the parameters

dataset_id='ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3'
stable_version = "2024-07-01"  # Adjust to the version you want

In [3]:
# Open the cell census
with cellxgene_census.open_soma() as census:
    # Step 1: Filter observation metadata by dataset_id
    obs_df = cellxgene_census.get_obs(
        census=census,
        organism="homo_sapiens",
        value_filter=f"dataset_id == '{dataset_id}'"
    )
    print(f"Retrieved {obs_df.shape[0]} observations (cells).")
    print(obs_df.head())
    
    # Step 2: Retrieve expression data for filtered cells as an AnnData object
    adata = cellxgene_census.get_anndata(
        census=census,
        organism="homo_sapiens",
        X_name="raw",  # Fetch raw counts
        obs_value_filter=f"dataset_id == '{dataset_id}'"
    )   
    print(f"Retrieved the expression data in an AnnData object `{adata}`")

    # Step 2: Use soma_joinid from adata.var to filter the var metadata
    soma_joinids = adata.var['soma_joinid'].astype(int).to_list()

    # Fetch var metadata with the correct coordinates
    var_df = cellxgene_census.get_var(
        census=census,
        organism="homo_sapiens",
        coords=soma_joinids
    )

    print(f"Retrieved {var_df.shape[0]} variable (gene) metadata records after filtering.")
    print(var_df.head())

The "stable" release is currently 2024-07-01. Specify 'census_version="2024-07-01"' in future calls to open_soma() to ensure data consistency.


Retrieved 836148 observations (cells).
   soma_joinid                            dataset_id      assay  \
0     40234926  ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3  10x 5' v1   
1     40234927  ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3  10x 5' v1   
2     40234928  ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3  10x 5' v1   
3     40234929  ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3  10x 5' v1   
4     40234930  ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3  10x 5' v1   

  assay_ontology_term_id                        cell_type  \
0            EFO:0011025              natural killer cell   
1            EFO:0011025  CD8-positive, alpha-beta T cell   
2            EFO:0011025                       blood cell   
3            EFO:0011025           non-classical monocyte   
4            EFO:0011025               classical monocyte   

  cell_type_ontology_term_id       development_stage  \
0                 CL:0000623  5-year-old human stage   
1                 CL:0000625  5-year-old human stage   
2                 CL:00

In [4]:
# Check if 'feature_id' or 'feature_name' contains the gene identifiers

print(adata.var[['soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs']].head(10))
print(adata.obs.head(10))


   soma_joinid       feature_id feature_name  feature_length       nnz  \
0            0  ENSG00000000003       TSPAN6            4530   4530448   
1            1  ENSG00000000005         TNMD            1476    236059   
2            2  ENSG00000000419         DPM1            9276  17576462   
3            3  ENSG00000000457        SCYL3            6883   9117322   
4            4  ENSG00000000460     C1orf112            5970   6287794   
5            5  ENSG00000000938          FGR            3382   5667858   
6            6  ENSG00000000971          CFH           15284   4423029   
7            7  ENSG00000001036        FUCA2            2822  10129388   
8            8  ENSG00000001084         GCLC            8618  12662710   
9            9  ENSG00000001167         NFYA            6209   4866873   

   n_measured_obs  
0        73855064  
1        61201828  
2        74159149  
3        73988868  
4        73636201  
5        74061500  
6        74095467  
7        73988868  
8    

In [5]:
# Manually filter the obs dataframe by dataset_id
adata = adata[adata.obs['dataset_id'] == dataset_id]


In [6]:
print(f"Unique diseases: {adata.obs['disease'].unique()}")

Unique diseases: ['COVID-19', 'normal', 'influenza']
Categories (3, object): ['COVID-19', 'influenza', 'normal']


In [7]:
# Check the type of adata.X
print(f"Type of adata.X: {type(adata.X)}")

# To view a small portion of the sparse matrix (e.g., the first 5 rows and columns)
dense_sample = adata.X[:5, :5].toarray()  # Convert a slice of the sparse matrix to dense
print(dense_sample)


Type of adata.X: <class 'anndata._core.views.SparseCSRView'>
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [ ]:
# Store raw counts in layers (since adata.X is already populated)
adata.layers['counts'] = adata.X
print("Raw counts stored in adata.layers['counts']")
# Store raw counts in layers


/var/folders/30/3sg2nxpx01n8zhgzmfp539mh0000gn/T/ipykernel_4447/3994753574.py:2: ImplicitModificationWarning: Setting element `.layers['counts']` of view, initializing view as actual.
  adata.layers['counts'] = adata.X


## Loading additional packages

First, we need to load the relevant packages, `scanpy` to handle scRNA-seq data
and `decoupler` to use statistical methods.


In [ ]:
import decoupler as dc
import scanpy as sc

# Only needed for processing
import numpy as np
import pandas as pd

# needed for writing h5ad objects
import json

# Needed for some plotting
import matplotlib.pyplot as plt

# Plotting options, change to your liking
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))

## Processing

Cellxgene data has already associated the gene symbols.  So we do not need to do that work. 
To be able to use `decoupler` we need to transform them into gene symbols:



###  Remove unannotated cells.

Since the meta-data of this data-set is available, we can filter cells that were not annotated:

In [ ]:
# Remove non-annotated cells
adata = adata[~adata.obs['cell_type'].isnull()]
# See how many remaining cells are there that have annotation 
print(f"Remaining annotated data in an AnnData object `{adata}`")


### How many cell types 

We would like to know how many unique cell types we have.

In [ ]:
print(f"Unique cell types: {adata.obs['cell_type'].unique()}")


We have 17 unique cell types in our data set.  

In [ ]:
# Normalize and log-transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers['normalized'] = adata.X

### Large sample needs reduction
Let's inspect how many cells are in each of the cell types for each of the donors


In [ ]:
# Step 1: Identify the donor_id and cell_type combinations with zero cell counts
zero_count_types = cell_type_counts[cell_type_counts['cell_count'] == 0][['donor_id', 'cell_type']]
print(f"Number of zero-count cell types: {zero_count_types.shape[0]}")
print("First 5 zero-count cell types:")
print(zero_count_types.head())

# Step 2: Convert the zero_count_types dataframe into a set of tuples (donor_id, cell_type)
zero_count_set = set(zip(zero_count_types['donor_id'], zero_count_types['cell_type']))
print(f"Size of zero_count_set: {len(zero_count_set)}")
print("First 5 items in zero_count_set:")
print(list(zero_count_set)[:5])

# Step 3: Create a boolean mask to filter out rows in adata.obs that have zero-count cell types
def is_non_zero_cell(row):
    return (row['donor_id'], row['cell_type']) not in zero_count_set

# Apply the mask creation function
non_zero_mask = adata.obs.apply(is_non_zero_cell, axis=1)

# Print the size of the mask and the first few values to verify it's being generated correctly
print(f"Size of the non_zero_mask: {non_zero_mask.size}")
print(f"First 10 values of the mask: {non_zero_mask[:10]}")

# Step 4: Apply the mask to filter adata
adata_filtered = adata[non_zero_mask].copy()

# Step 5: Print the new size of adata after filtering to validate the process
print(f"New size of adata_filtered: {adata_filtered.shape}")


In [ ]:
# Identify the donor_id and cell_type combinations with zero cell counts
zero_count_types = cell_type_counts[cell_type_counts['cell_count'] == 0][['donor_id', 'cell_type']]
print(f"Number of zero-count cell types: {zero_count_types.shape[0]}")
print(zero_count_types.head())

# Convert the zero_count_types dataframe into a set of tuples (donor_id, cell_type)
zero_count_set = set(zip(zero_count_types['donor_id'], zero_count_types['cell_type']))
print(f"Size of zero_count_set: {len(zero_count_set)}")

# Create a boolean mask to filter out rows in adata.obs that have zero-count cell types
non_zero_mask = adata.obs.apply(lambda row: (row['donor_id'], row['cell_type']) not in zero_count_set, axis=1)
print(f"Size of the non_zero_mask: {non_zero_mask.size}")
print(f"First 10 values of the mask: {non_zero_mask[:10]}")

# Apply the mask to filter adata
adata_filtered = adata[non_zero_mask].copy()

# Print the new size of adata after filtering
print(f"New size of adata_filtered: {adata_filtered.shape}")


In [ ]:
# Group by donor_id and cell_type, and count the cells in each group
cell_type_counts = adata.obs.groupby(['donor_id', 'cell_type']).size().reset_index(name='cell_count')
# Print the results
print(f"Cell_type_counts after cell_type filtering:`{cell_type_counts[['donor_id', 'cell_type', 'cell_count']]}`")

# See how many remaining cells are there that have annotation 
print(f"Remaining annotated data in an AnnData object after cell_type filtering `{adata}`")


In [ ]:
# Scale the data
sc.pp.scale(adata, max_value=10)


In [ ]:
# Generate PCA features
sc.tl.pca(adata, svd_solver='arpack', use_highly_variable=True)



In [ ]:
# Identify highly variable genes
sc.pp.highly_variable_genes(adata, batch_key='donor_id')
# Check the type of sc.pp.highly_variable_genes
print(f"Type of sc.pp.highly_variable_genes: {type(sc.pp.highly_variable_genes)}")

In [ ]:
# Compute distances in the PCA space, and find cell neighbors
sc.pp.neighbors(adata)


In [ ]:

# Generate UMAP features
sc.tl.umap(adata)


In [ ]:

# Visualize
sc.pl.umap(adata, color=['disease','cell_type'], frameon=False)

In this data-set we have two condition, `COVID-19` and `healthy`, across 6 different cell types.

## Generation of pseudo-bulk profiles

After the annotation of clusters into cell identities, we often would like to perform differential expression analysis (DEA) between conditions within particular cell types to further characterize them. DEA can be performed at the single-cell level, but the obtained p-values are often inflated as each cell is treated as a sample. We know that single cells within a sample are not independent of each other, since they were isolated from the same environment. If we treat cells as samples, we are not testing the variation across a population of samples, rather the variation inside an individual one. Moreover, if a sample has more cells than another it might bias the results. 

The current best practice to correct for this is using a pseudo-bulk approach ([Squair J.W., et al 2021](https://doi.org/10.1038/s41467-021-25960-2)), which involves the following steps:

1. Subsetting the cell type(s) of interest to perform DEA.
2. Extracting their raw integer counts.
3. Summing their counts per gene into a single profile if they pass quality control.
4. Performing DEA if at least two biological replicates per condition are available (more replicates are recommended).

We can pseudobulk using the function `decoupler.get_pseudobulk`. In this example, we are interested in summing the counts but other
modes are available, for more information check its argument `mode`.

In [ ]:
# Get pseudo-bulk profile
pdata = dc.get_pseudobulk(
    adata,
    sample_col='individual',
    groups_col='cell_type',
    layer='counts',
    mode='sum',
    min_cells=0,
    min_counts=0
)

It has generated a profile for each sample and cell type. We can plot their quality control metrics:

In [ ]:
dc.plot_psbulk_samples(pdata, groupby=['individual', 'cell_type'], figsize=(12, 4))

There are two criteria to filter low quality samples: its number of cells (`psbulk_n_cells`), and its total sum of counts (`psbulk_counts`).
In these plots it can be seen that there are some samples of platelet cells that contain less than 10 cells, we might want to remove
them by using the arguments `min_cells` and `min_counts`. Note that these thresholds are arbitrary and will change depening on the
dataset, but a good rule of thumb is to have at least 10 cells with 1000 accumulated counts.

In [ ]:
# Get filtered pseudo-bulk profile
pdata = dc.get_pseudobulk(
    adata,
    sample_col='individual',
    groups_col='cell_type',
    layer='counts',
    mode='sum',
    min_cells=10,
    min_counts=1000
)
pdata

### Exploration of pseudobulk profiles
Now that we have generated the pseudobulk profiles for each patient and each cell type, let's explore the variability between them. For that, we will first do some simple preprocessing and then do a PCA

In [ ]:
# Store raw counts in layers
pdata.layers['counts'] = pdata.X.copy()

# Normalize, scale and compute pca
sc.pp.normalize_total(pdata, target_sum=1e4)
sc.pp.log1p(pdata)
sc.pp.scale(pdata, max_value=10)
sc.tl.pca(pdata)

# Return raw counts to X
dc.swap_layer(pdata, 'counts', X_layer_key=None, inplace=True)

In [ ]:
sc.pl.pca(pdata, color=['disease', 'cell_type'], ncols=1, size=300)
sc.pl.pca_variance_ratio(pdata)

When looking at the PCA, it seems like the two first components explain most of the variance and they easily separate cell types from one another. In contrast, the principle components do not seem to be associated with disease status as such.

In order to have a better overview of the association of PCs with sample metadata, let's perform ANOVA on each PC and see whether they are significantly associated with any technical or biological annotations of our samples

In [ ]:
dc.get_metadata_associations(
    pdata,
    obs_keys = ['sex', 'disease', 'cell_type', 'psbulk_n_cells', 'psbulk_counts'],  # Metadata columns to associate to PCs
    obsm_key='X_pca',  # Where the PCs are stored
    uns_key='pca_anova',  # Where the results are stored
    inplace=True,
)

In [ ]:
dc.plot_associations(
    pdata,
    uns_key='pca_anova',  # Summary statistics from the anova tests
    obsm_key='X_pca',  # where the PCs are stored
    stat_col='p_adj',  # Which summary statistic to plot
    obs_annotation_cols = ['disease', 'cell_type'], # which sample annotations to plot
    titles=['Principle component scores', 'Adjusted p-values from ANOVA'],
    figsize=(7, 5),
    n_factors=10,
)

On the PCA plots above, T and B cells seemed not to be that well separated. However when looking at the hierarchical clustering in the heatmap, one can see that the inclusion of more PCs helps to distinguish them.

When looking at the p-values from the ANOVA models, it becomes clear that the top PCs, which explain most of the observed variability, are statistically associated with the `cell_type` category.

### Pseudo-bulk profile gene filtering
Additionally to filtering low quality samples, we can also filter noisy expressed genes in case we want to perform downstream analyses such as DEA afterwards. Note that this step should be done at the cell type level, since each cell type may express different collection of genes.

For this vignette, we will explore the effects of COVID on T cells. Let's first select our samples of interest:

In [ ]:
# Select T cell profiles
tcells = pdata[pdata.obs['cell_type'] == 'T cell'].copy()

To filter genes, we will follow the strategy implemented in the function `filterByExpr` from [edgeR](https://rdrr.io/bioc/edgeR/man/filterByExpr.html).
It keeps genes that have a minimum total number of reads across samples (`min_total_count`), and that have a minimum number of counts in a number of samples (`min_count`).

We can plot how many genes do we keep, you can play with the `min_count` and `min_total_count` to check how many genes would be kept when changed:

In [ ]:
dc.plot_filter_by_expr(tcells, group='disease', min_count=10, min_total_count=15)

Here we can observe the frequency of genes with different total sum of counts and number of samples. The dashed lines indicate the current thresholds, meaning that only the genes in the upper-right corner are going to be kept. Filtering parameters is completely arbitrary, but a good rule of thumb is to identify bimodal distributions and split them modifying the available thresholds.
In this example, with the default values we would keep a good quantity of genes while filtering potential noisy genes.

<div class="alert alert-info">

**Note**
    
Changing the value of `min_count` will drastically change the distribution of "Number of samples", not change its threshold.
In case you want to lower or increase it, you need to play with the `group`, `large_n` and `min_prop` parameters. 


</div>

Once we are content with the threshold parameters, we can perform the actual filtering:

In [ ]:
# Obtain genes that pass the thresholds
genes = dc.filter_by_expr(tcells, group='disease', min_count=10, min_total_count=15)

# Filter by these genes
tcells = tcells[:, genes].copy()
tcells

In [ ]:
print(type(tcells.X))
print(tcells.X.shape)

In [ ]:
for key, value in tcells.uns.items():
    print(f"Key: {key}, Type: {type(value)}")

In [ ]:
# Function to handle and split mixed dtype NumPy array (strings and numbers)
def split_mixed_array(mixed_array):
    """Split a NumPy array with mixed types into numeric and string parts."""
    string_columns = mixed_array[:, [0, 3]]  # Extract the columns containing strings
    numeric_columns = mixed_array[:, [1, 2, 4]]  # Extract the numeric columns
    
    # Convert numeric columns to float64 (forcing errors to NaN for robustness)
    numeric_columns = numeric_columns.astype(np.float64)
    
    return string_columns, numeric_columns

# Function to handle DataFrame, separating numeric and non-numeric columns
def split_dataframe(df):
    """Split a DataFrame into numeric and non-numeric parts."""
    # Select numeric columns and convert them to float64
    numeric_cols = df.select_dtypes(include=[np.number]).astype(np.float64)
    
    # Select non-numeric columns (object, category, etc.)
    non_numeric_cols = df.select_dtypes(exclude=[np.number])
    
    return numeric_cols, non_numeric_cols

# Function to recursively handle dictionaries, ensuring NumPy arrays are left intact
def handle_dict(value):
    clean_dict = {}
    for sub_key, sub_value in value.items():
        if isinstance(sub_value, np.ndarray):
            print(f"Keeping NumPy array inside dict at key '{sub_key}' as is.")
            clean_dict[sub_key] = sub_value  # Leave NumPy arrays unchanged
        elif isinstance(sub_value, dict):
            clean_dict[sub_key] = handle_dict(sub_value)  # Recursively clean sub-dictionaries
        else:
            clean_dict[sub_key] = sub_value  # Keep other types as is
    return clean_dict

# Create a new uns_fixed dictionary to store the fixed entries
uns_fixed = {}

# Iterate over the uns items and fix problematic types
for key, value in tcells.uns.items():
    # Handle NumPy arrays directly (no conversion needed for HDF5)
    if isinstance(value, np.ndarray):
        print(f"Keeping NumPy array at key '{key}' as is.")
        
        if key == "pca_anova_data":
            # Split mixed array into string and numeric parts
            string_cols, numeric_cols = split_mixed_array(value)
            print(f"Storing separated string columns and numeric data for '{key}'")
            
            # Store numeric and string data separately
            uns_fixed[f'{key}_numeric'] = numeric_cols
            uns_fixed[f'{key}_strings'] = string_cols
        
        else:
            uns_fixed[key] = value  # No change for other NumPy arrays
    
    # Handle pandas DataFrame by separating numeric and non-numeric columns
    elif isinstance(value, pd.DataFrame):
        print(f"Processing DataFrame at key '{key}'")
        
        # Split DataFrame into numeric and non-numeric parts
        numeric_part, non_numeric_part = split_dataframe(value)
        
        # Store numeric part as NumPy array
        if not numeric_part.empty:
            uns_fixed[f'{key}_data'] = numeric_part.values.astype(np.float64)
        
        # Store non-numeric part as strings (index and columns)
        if not non_numeric_part.empty:
            uns_fixed[f'{key}_non_numeric'] = non_numeric_part.values.astype(str)
        
        # Store index and columns as strings
        uns_fixed[f'{key}_index'] = value.index.values.astype(str)
        uns_fixed[f'{key}_columns'] = value.columns.values.astype(str)
    
    # Handle nested dictionaries without JSON conversion
    elif isinstance(value, dict):
        print(f"Handling dict at key '{key}' without JSON serialization.")
        clean_dict = handle_dict(value)  # Recursively handle NumPy arrays inside the dict
        uns_fixed[key] = clean_dict  # Keep the cleaned dict as is

    # For other types (like strings, floats), keep them unchanged
    else:
        print(f"Keeping key '{key}' of type '{type(value)}' unchanged.")
        uns_fixed[key] = value

# Replace the original uns with the fixed version
tcells.uns = uns_fixed

# Now attempt to write the AnnData object to an .h5ad file
tcells.write("tcells.h5ad")

Another filtering strategy is to filter out genes that are not expressed in a percentage of cells and samples, as implemented
in `decoupler.filter_by_prop`.

## Contrast between conditions
Once we have generated robust pseudo-bulk profiles, we can compute DEA. For this example, we will perform a simple
experimental design where we compare the gene expression of T cells from diseased patients against controls. We will use the
python implementation of the framework DESeq2, but we could have used any other one (`limma` or `edgeR` for example).
For a better understanding how it works, check [DESeq2's documentation](https://pydeseq2.readthedocs.io/en/latest/). Note that
more complex experimental designs can be used by adding more factors to the `design_factors` argument.

In [ ]:
# Import DESeq2
from pydeseq2.dds import DeseqDataSet, DefaultInference
from pydeseq2.ds import DeseqStats

In [ ]:
# Build DESeq2 object
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    adata=tcells,
    design_factors='disease',
    ref_level=['disease', 'normal'],
    refit_cooks=True,
    inference=inference,
)

In [ ]:
# Compute LFCs
dds.deseq2()

In [ ]:
# Extract contrast between COVID-19 vs normal
stat_res = DeseqStats(
    dds,
    contrast=["disease", 'COVID-19', 'normal'],
    inference=inference,
)

In [ ]:
# Compute Wald test
stat_res.summary()

In [ ]:
# Extract results
results_df = stat_res.results_df
results_df
results_df.to_parquet("results_df.parquet")

We can plot the obtained results in a volcano plot:

In [ ]:
dc.plot_volcano_df(
    results_df,
    x='log2FoldChange',
    y='padj',
    top=20,
    figsize=(8, 4)
)

After performing DEA, we can use the obtained gene level statistics to perform enrichment analysis. Any statistic can be used,
but we recommend using the t-values instead of logFCs since t-values incorporate the significance of change in their value.
We will transform the obtained t-values stored in `stats` to a wide matrix so that it can be used by `decoupler`:

In [ ]:
mat = results_df[['stat']].T.rename(index={'stat': 'T cell'})
mat

## Transcription factor activity inference

The first functional analysis we can perform is to infer transcription factor (TF) activities from our transcriptomics data. We will need a gene regulatory network (GRN) and a statistical method.

### CollecTRI network
[CollecTRI](https://github.com/saezlab/CollecTRI) is a comprehensive resource
containing a curated collection of TFs and their transcriptional targets
compiled from 12 different resources. This collection provides an increased
coverage of transcription factors and a superior performance in identifying
perturbed TFs compared to our previous
[DoRothEA](https://saezlab.github.io/dorothea/) network and other literature
based GRNs. Similar to DoRothEA, interactions are weighted by their mode of
regulation (activation or inhibition).

For this example we will use the human version (mouse and rat are also
available). We can use `decoupler` to retrieve it from `omnipath`. The argument
`split_complexes` keeps complexes or splits them into subunits, by default we
recommend to keep complexes together.

<div class="alert alert-info">

**Note**

In this tutorial we use the network CollecTRI, but we could use any other GRN coming from an inference method such as [CellOracle](https://morris-lab.github.io/CellOracle.documentation/), [pySCENIC](https://pyscenic.readthedocs.io/en/latest/) or [SCENIC+](https://scenicplus.readthedocs.io/en/latest/). 

</div> 

In [ ]:
# Retrieve CollecTRI gene regulatory network
collectri = dc.get_collectri(organism='human', split_complexes=False)
collectri

### Activity inference with Univariate Linear Model (ULM)

To infer TF enrichment scores we will run the Univariate Linear Model (`ulm`) method. For each sample in our dataset (`mat`) and each TF in our network (`net`), it fits a linear model that predicts the observed gene expression
based solely on the TF's TF-Gene interaction weights. Once fitted, the obtained t-value of the slope is the score. If it is positive, we interpret that the TF is active and if it is negative we interpret that it is inactive.

<img src="../ulm.png" />

We can run `ulm` with a one-liner:

In [ ]:
# Infer pathway activities with ulm
tf_acts, tf_pvals = dc.run_ulm(mat=mat, net=collectri)
tf_acts

Let us plot the obtained scores for the top active/inactive transcription factors:

In [ ]:
dc.plot_barplot(
    acts=tf_acts,
    contrast='T cell',
    top=25,
    vertical=True,
    figsize=(3, 6)
)

In accordance to the previous pathway results, T cells seem to activate for TFs responsible for cell growth (E2F4, TFDP1, E2F1).

Like with pathways, we can explore how the target genes look like:

In [ ]:
# Extract logFCs and pvals
logFCs = results_df[['log2FoldChange']].T.rename(index={'log2FoldChange': 'T cell'})
pvals = results_df[['padj']].T.rename(index={'padj': 'T cell'})

# Plot
dc.plot_volcano(
    logFCs=logFCs,
    pvals=pvals,
    contrast='T cell',
    name='E2F4',
    net=collectri,
    top=10,
    sign_thr=0.05,
    lFCs_thr=0.5
)

We can also plot the network of interesting TFs (top and bottom by activity) and color the nodes by activity and target gene expression:

In [ ]:
dc.plot_network(
    net=collectri,
    obs=mat,
    act=tf_acts,
    n_sources=['MYC', 'E2F4', 'HSF1', 'GATA6'],
    n_targets=15,
    node_size=100,
    figsize=(7, 7),
    c_pos_w='darkgreen',
    c_neg_w='darkred',
    vcenter=True
)

Green edges are positive regulation (activation), red edges are negative regulation (inactivation):

## Pathway activity inference

Another analysis we can perform is to infer pathway activities from our transcriptomics data.

### PROGENy model

[PROGENy](https://saezlab.github.io/progeny/) is a comprehensive resource containing a curated collection of pathways and their target genes, with weights for each interaction.
For this example we will use the human weights (other organisms are available) and we will use the top 500 responsive genes ranked by p-value. Here is a brief description of each pathway:

- **Androgen**: involved in the growth and development of the male reproductive organs.
- **EGFR**: regulates growth, survival, migration, apoptosis, proliferation, and differentiation in mammalian cells
- **Estrogen**: promotes the growth and development of the female reproductive organs.
- **Hypoxia**: promotes angiogenesis and metabolic reprogramming when O2 levels are low.
- **JAK-STAT**: involved in immunity, cell division, cell death, and tumor formation.
- **MAPK**: integrates external signals and promotes cell growth and proliferation.
- **NFkB**: regulates immune response, cytokine production and cell survival.
- **p53**: regulates cell cycle, apoptosis, DNA repair and tumor suppression.
- **PI3K**: promotes growth and proliferation.
- **TGFb**: involved in development, homeostasis, and repair of most tissues.
- **TNFa**: mediates haematopoiesis, immune surveillance, tumour regression and protection from infection.
- **Trail**: induces apoptosis.
- **VEGF**: mediates angiogenesis, vascular permeability, and cell migration.
- **WNT**: regulates organ morphogenesis during development and tissue repair.

To access it we can use `decoupler`.

In [ ]:
# Retrieve PROGENy model weights
progeny = dc.get_progeny(top=500)
progeny

### Activity inference with Multivariate Linear Model (MLM)

To infer pathway enrichment scores we will run the Multivariate Linear Model (`mlm`) method. For each sample in our dataset (`adata`), it fits a linear model that predicts the observed gene expression based on all pathways' Pathway-Gene interactions weights.
Once fitted, the obtained t-values of the slopes are the scores. If it is positive, we interpret that the pathway is active and if it is negative we interpret that it is inactive.

<img src="../mlm.png" />
     
We can run `mlm` with a one-liner:

In [ ]:
# Infer pathway activities with mlm
pathway_acts, pathway_pvals = dc.run_mlm(mat=mat, net=progeny)
pathway_acts

Let us plot the obtained scores:

In [ ]:
dc.plot_barplot(
    acts=pathway_acts,
    contrast='T cell',
    top=25,
    vertical=False,
    figsize=(6, 3)
)

It looks like JAK-STAT, a known immunity pathway is more active in T cells from COVID-19 patients than in controls. To further explore how the target genes of a pathway of interest behave, we can plot them in scatter plot:

In [ ]:
dc.plot_targets(
    data=results_df,
    stat='stat',
    source_name='JAK-STAT',
    net=progeny,
    top=15
)

The observed activation of JAK-STAT is due to the fact that majority of its target genes with positive weights have positive
t-values (1st quadrant), and the majority of the ones with negative weights have negative t-values (3d quadrant).

## Functional enrichment of biological terms

Finally, we can also infer activities for general biological terms or processes.

### MSigDB gene sets

The Molecular Signatures Database ([MSigDB](http://www.gsea-msigdb.org/gsea/msigdb/)) is a resource containing a collection of gene sets annotated to different biological processes.

In [ ]:
# Retrieve MSigDB resource
msigdb = dc.get_resource('MSigDB')
msigdb

As an example, we will use the hallmark gene sets, but we could have used any other. 

<div class="alert alert-info">

**Note**
    
To see what other collections are available in MSigDB, type: `msigdb['collection'].unique()`.

</div>  

We can filter by for `hallmark`:

In [ ]:
# Filter by hallmark
msigdb = msigdb[msigdb['collection']=='hallmark']

# Remove duplicated entries
msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]

# Rename
msigdb.loc[:, 'geneset'] = [name.split('HALLMARK_')[1] for name in msigdb['geneset']]

msigdb

### Enrichment with Over Representation Analysis (ORA)

To infer functional enrichment scores we will run the Over Representation Analysis (`ora`) method.
As input data it accepts an expression matrix (`decoupler.run_ora`) or the results of differential expression analysis (`decoupler.run_ora_df`).
For the former, by default the top 5% of expressed genes by sample are selected as the set of interest (S*), and for the latter a user-defined
significance filtering can be used.
Once we have S*, it builds a contingency table using set operations for each set stored in the gene set resource being used (`net`).
Using the contingency table, `ora` performs a one-sided Fisher exact test to test for significance of overlap between sets.
The final score is obtained by log-transforming the obtained p-values, meaning that higher values are more significant.

<img src="../ora.png" />
     
We can run `ora` with a simple one-liner:

In [ ]:
# Infer enrichment with ora using significant deg
top_genes = results_df[results_df['padj'] < 0.05]

# Run ora
enr_pvals = dc.get_ora_df(
    df=top_genes,
    net=msigdb,
    source='geneset',
    target='genesymbol'
)

enr_pvals.head()

Then we can visualize the most enriched terms:

In [ ]:
dc.plot_dotplot(
    enr_pvals.sort_values('Combined score', ascending=False).head(15),
    x='Combined score',
    y='Term',
    s='Odds ratio',
    c='FDR p-value',
    scale=0.5,
    figsize=(3, 9)
)

We can also plot the running score for a given gene set:

In [ ]:
# Plot
dc.plot_running_score(
    df=results_df,
    stat='stat',
    net=msigdb,
    source='geneset',
    target='genesymbol',
    set_name='MYC_TARGETS_V1'
)